# Data ETL pipeline

In [1]:
import sys
sys.path.append("../")
from src.service.provider import ProviderService
from src.prepare.data_etl import DataETL
import pandas as pd

provider = ProviderService()
# fix this
config = provider.categories.full_data

DATA_FOLDER = config.folder_name
SIZE = config.size
OVERLAP = config.overlap

# provider.config.enable_tracing(project="ETL")
etl = DataETL(provider=provider, rag_config=config)

In [2]:

a = etl.try_split_text(data_folder=DATA_FOLDER, size=SIZE, overlap=OVERLAP, use_sitemap=True)

From (59, 5) to Counter({3: 22, 2: 12, 4: 9, 5: 5, 1: 5, 7: 3, 6: 2, 8: 1}) -> 197


## Reset 

In [3]:
etl.reset_config_repo(provider=provider, rag_config=config)

DONE


## Upload docs

In [4]:
## OLD
a = etl.upload_df_docs(
    data_folder=DATA_FOLDER,
    size=SIZE,
    overlap=OVERLAP, re_update=False)

In [4]:
config.vector_index

'full-data-vector'

In [5]:
a = etl.upload_from_sitemap(
    data_folder=DATA_FOLDER, 
    size=SIZE, overlap=OVERLAP, re_update=True)

-> DONE: if upload to elastic search 3 & 3 docs
-> DONE: upload to firebase at full_data
-> DONE: if upload to elastic search 4 & 4 docs
-> DONE: upload to firebase at full_data
-> DONE: if upload to elastic search 3 & 3 docs
-> DONE: upload to firebase at full_data
-> DONE: if upload to elastic search 3 & 3 docs
-> DONE: upload to firebase at full_data
-> DONE: if upload to elastic search 5 & 5 docs
-> DONE: upload to firebase at full_data
-> DONE: if upload to elastic search 6 & 6 docs
-> DONE: upload to firebase at full_data
-> DONE: if upload to elastic search 3 & 3 docs
-> DONE: upload to firebase at full_data
-> DONE: if upload to elastic search 4 & 4 docs
-> DONE: upload to firebase at full_data
-> DONE: if upload to elastic search 5 & 5 docs
-> DONE: upload to firebase at full_data
-> DONE: if upload to elastic search 3 & 3 docs
-> DONE: upload to firebase at full_data
-> DONE: if upload to elastic search 3 & 3 docs
-> DONE: upload to firebase at full_data
-> DONE: if upload to

In [6]:
print(a[-1].content)

A table of content
1. Ngành Bảo hộ lao động
2. Ngành Công nghệ sinh học
3. Ngành Công tác xã hội
4. Ngành Dược
5. Ngành Kế toán
6. Ngành Khoa học máy tính
7. Ngành Khoa học môi trường
8. Ngành Kiến trúc
9. Ngành Kinh doanh quốc tế
10. Ngành Kỹ thuật cơ điện tử
11. Ngành Kỹ thuật hóa học
12. Ngành Kỹ thuật phần mềm
13. Ngành Kỹ thuật Xây dựng
14. Ngành Kỹ thuật xây dựng công trình giao thông
15. Ngành Kỹ thuật điện
16. Ngành Kỹ thuật Điện tử - Viễn thông
17. Ngành Kỹ thuật điều khiển và tự động hóa
18. Ngành Luật
19. Ngành mạng máy tính và truyền thông dữ liệu
20. Ngành Marketing
21. Ngành ngôn ngữ Anh
22. Ngành ngôn ngữ Trung Quốc
23. Ngành quan hệ lao động
24. Ngành quản lý thể dục thể thao - Chuyên ngành Golf
25. Ngành quản lý thể dục thể thao - Chuyên ngành kinh doanh thể thao và tổ chức sự kiện
26. Ngành quản lý xây dựng
27. Ngành quản trị kinh doanh - Chuyên ngành Quản trị nguồn nhân lực
28. Ngành quản trị kinh doanh - Chuyên ngành Quản trị nhà hàng và khách sạn
29. Ngành Quy hoạc

In [4]:
len(a)
etl.index_page

'trường đại học Tôn Đức Thắng: Tiền thân của Trường...\ntrường đại học Tôn Đức Thắng: \nSỨ MẠNG của trường ...\n'

In [5]:
etl.delete_doc(a[0])

In [6]:
etl.delete_doc(a[1])

## Upload 1 doc

In [7]:
category = "test_qa"

etl.upload_doc(
    db_category=category,
    data=df.iloc[1],
    text_splitter=etl.create_splitter(), 
    es=provider.get_elasticsearch_store(index=category))

TypeError: upload_doc() got an unexpected keyword argument 'db_category'

In [ ]:
es = provider.get_elasticsearch_store(index=category)
a = es.similarity_search(query="bye")
a

[Document(page_content='Bye is a common phrase for departing', metadata={'id': '8af20725-b2e8-4f04-b5ce-0567de3a64ca'})]

In [ ]:
id = a[0].metadata['id']
a = etl.fire.read_document(collection=category, document_id=id)
a

In [ ]:
a.content

' Bye is a common phrase for departing'